In [1]:
!pip install -q gdown --no-use-pep517

In [3]:
!pip install tensorflow_hub

In [4]:
import os

import tensorflow as tf
import tensorflow_hub as hub

import gdown
import numpy as np
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns

### About the dataset

We will use BARD (Bangla Article Dataset) which has around 3,76,226 articles collected from different Bangla news portals and labelled with 5 categories : economy, state, international, sports and entertainment.

In [5]:
gdown.download(
    url='https://drive.google.com/uc?id=1Ag0jd21oRwJhVFIBohmX_ogeojVtapLy',
    output='bard.zip',
    quiet=True
)

'bard.zip'

In [7]:
!curl -O https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.bn.300.vec.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0  840M    0 35719    0     0  15014      0 16:18:16  0:00:02 16:18:14 15014
  0  840M    0 1672k    0     0   543k      0  0:26:23  0:00:03  0:26:20  543k
  0  840M    0 5846k    0     0  1416k      0  0:10:07  0:00:04  0:10:03 1415k
  1  840M    1 13.3M    0     0  2700k      0  0:05:18  0:00:05  0:05:13 2774k
  2  840M    2 21.6M    0     0  3627k      0  0:03:57  0:00:06  0:03:51 5107k
  2  840M    2 24.9M    0     0  3615k      0  0:03:58  0:00:07  0:03:51 5439k
  3  840M    3 27.9M    0     0  3544k      0  0:04:02  0:00:08  0:03:54 5392k
  3  840M    3 30.9M    0     0  3491k      0  0:04

In [8]:
! curl -O https://raw.githubusercontent.com/tensorflow/hub/master/examples/text_embeddings_v2/export_v2.py

 51  840M   51  433M    0     0  4570k      0  0:03:08  0:01:37  0:01:31 5161k
 51  840M   51  436M    0     0  4559k      0  0:03:08  0:01:38  0:01:30 4403k
 52  840M   52  440M    0     0  4547k      0  0:03:09  0:01:39  0:01:30 4421k
 52  840M   52  444M    0     0  4545k      0  0:03:09  0:01:40  0:01:29 4554k
 53  840M   53  451M    0     0  4577k      0  0:03:08  0:01:41  0:01:27 4970k
 54  840M   54  455M    0     0  4568k      0  0:03:08  0:01:42  0:01:26 4537k
 54  840M   54  459M    0     0  4561k      0  0:03:08  0:01:43  0:01:25 4614k
 55  840M   55  463M    0     0  4556k      0  0:03:08  0:01:44  0:01:24 4719k
 55  840M   55  469M    0     0  4577k      0  0:03:08  0:01:45  0:01:23 5222k
 56  840M   56  474M    0     0  4577k      0  0:03:07  0:01:46  0:01:21 4588k
 56  840M   56  478M    0     0  4571k      0  0:03:08  0:01:47  0:01:21 4636k
 57  840M   57  482M    0     0  4567k      0  0:03:08  0:01:48  0:01:20 4684k
 58  840M   58  488M    0     0  4581k      0  0:03: